In [1]:
!pip install --upgrade tensorflow
!pip install pytz

In [2]:
import matplotlib.pyplot as plt
import importlib
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import product
from datetime import datetime
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import forgeall_ssae_model_manager
importlib.reload(forgeall_ssae_model_manager)
from forgeall_ssae_model_manager import *

import forgeall_ssae_model_inferencers
importlib.reload(forgeall_ssae_model_inferencers)
from forgeall_ssae_model_inferencers import *


#NEW HEA PREDICTIONS
Use the SSAE model to get the latent vector projections for new_dataset_normalized.csv
Then load the k nearest neighbor regressed model file for the above SSAE model
Then use the latent vector projections from above to predict the new HEA's

In [9]:
new_dataset_normalized = pd.read_csv('new_dataset_normalized.csv')

'''
ssae model and correspoinding knn pair files:
1)                                            2)                                             3)
Pair1-hea_sparse_autoencoder_model.keras      Pair2-03052025_08_36_30_ssae_model.keras       Pair3-sparse_autoencoder_model.keras
Pair1-03052025_21_14_27_best_knn_model.pkl    Pair2-03052025_08_38_19_best_knn_model.pkl     Pair3-03052025_21_26_40_best_knn_model.pkl

'''

ssae_model_file                   = './Pair1-hea_sparse_autoencoder_model.keras'
corresponding_best_knn_model_file = './Pair1-03052025_21_14_27_best_knn_model.pkl'

#---------- FIRST load the SSAE model
loaded_model, model_latent_layer_encoding = fa_load_model_from_file(ssae_model_file)
#--------- and then load the CORRESPONDING K Nearest Neighbor regressor file that generated using that model.
loaded_best_knn = fa_load_scikit_regressed_model_from_file(corresponding_best_knn_model_file)

#--------- SECOND get the latent vector projections (latent and latent_scaled) for the new dataset
(latent_test_new,
 latent_test_new_scaled)  = fa_get_latent_prediction_vectors_from_model(model_latent_layer_encoding,
                                                                        new_dataset_normalized
                                                                       )
#-------- THIRD, now predict the new HEA's
knn_predictions_test_new = loaded_best_knn.predict (latent_test_new_scaled)

#-----------FINAL OUTPUT--------------------------------
# Filter predictions for high yield strength (e.g., above a certain threshold)
df=pd.read_csv('new_dataset -fixed.csv')

# Assuming knn_predictions_test_new contains the predicted yield strengths
# Sort the predictions in descending order and get the indices of the top 2
# Flatten the predictions to a 1D array
flattened_predictions = knn_predictions_test_new.ravel()

# Sort the predictions in descending order and get the indices of the top 2
top_2_indices = np.argsort(flattened_predictions)[-2:][::-1]

print(knn_predictions_test_new)

# Retrieve the corresponding rows from the dataset
top_2_heas = df.iloc[top_2_indices]

top_2_heas.to_csv('top_2_heas.csv', index=False)
# Print the top 2 HEAs
print(top_2_heas)

#--------- Print the predictions at those indices ----
for idx in top_2_indices:
    print(f"Prediction at index {idx}: {flattened_predictions[idx]}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 25)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 16)             │            416 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ latent_layer (Dense)      │ (None, 6)              │            102 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 16)             │            112 │ latent_layer[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 25)             │            425 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 1)              │              7 │ latent_layer[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,062 (4.15 KB)

 Trainable params: 1,062 (4.15 KB)

 Non-trainable params: 0 (0.00 B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
[[0.38773727]
 [0.32877252]
 [0.22792023]
 ...
 [0.48933775]
 [0.39695443]
 [0.30335124]]
      C(at%)  Al(at%)  V(at%)  Cr(at%)  Mn(at%)  Fe(at%)  Co(at%)  Ni(at%)  \
1039       0    9.091   9.091   18.182    0.000   27.273    9.091    9.091   
1803       0   13.333   6.667   20.000    6.667    6.667   13.333   20.000   

      Cu(at%)  Mo(at%)  ...  Config_Entropy  (0-T)*Entropy  melting point(C)  \
1039    9.091    9.091  ...         16.3973     -4886.4080       1605.134233   
1803    6.667    6.667  ...         17.3246     -5162.7342       1519.927489   

      grain(um)  Anneal_Temp(K)  Anneal_Time(h)  R(%)  CR(%)  Hom_Temp(K)  \
1039    12.8914             893         24.0000  20.0   67.0            0   
1803    12.8914               0          0.0833  20.0   30.0            0   

      Enthalpy_FCC  
1039   -11210.0183  
1803   -11208.2106  

[2 rows x 25 columns]
Prediction at index 1039: 1.0
Prediction at index 1803: 0.9999999999999999
